In [7]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from numpy import linalg as ln #to compute eigen values/ vectors
from netCDF4 import Dataset

In [7]:
print(xr.backends.list_engines())

{'scipy': <ScipyBackendEntrypoint>
  Open netCDF files (.nc, .nc4, .cdf and .gz) using scipy in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.ScipyBackendEntrypoint.html, 'store': <StoreBackendEntrypoint>
  Open AbstractDataStore instances in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.StoreBackendEntrypoint.html}


In [8]:
SST1 = xr.open_dataset("saved_sst_data.nc") #, engine="nc4")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jessicasmith/Desktop/EAS6995/Lab0/saved_sst_data.nc'

In [12]:
#SST = xr.open_dataset("sst.mon.ltm.1991-2020.nc", engine="nc4")
SST = xr.open_dataset("sst.mon.ltm.1991-2020.nc", engine="scipy")

TypeError: Error: /Users/jessicasmith/Desktop/EAS6995/sst.mon.ltm.1991-2020.nc is not a valid NetCDF 3 file
            If this is a NetCDF4 file, you may need to install the
            netcdf4 library, e.g.,

            $ pip install netcdf4
            